# LSTM을 이용해 감성 분류기 만들기

In [1]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torchtext.datasets.imdb import IMDB
import sys

# 데이터 준비하기

이번에는 다음 코드처럼 텍스트의 최대 길이를 길게 설정(200)하고 batch_first 인수는 False로 설정했다.

In [2]:
TEXT = data.Field(lower=True,fix_length=200,batch_first=False)
LABEL = data.Field(sequential=False,)

In [3]:
train, test = IMDB.splits(TEXT, LABEL)

In [4]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

# 배치 처리기 생성하기

배치 데이터의 형상은 시퀀스 길이와 배치의 크기가 된다.

따라서 이번에 배치 데이터의 형상은 시퀀스 길이가 200이고 배치 크기는 32이므로 (200, 32)가 된다.

In [5]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1)
train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


# 네트워크 생성하기

forward 메소드는 크기 (200, 32)의 입력 데이터를 임베디드 레이어에 통과시킨다

배치의 각 토큰은 임베딩으로 대체되고 그 크기는 (200, 32, 100)으로 바뀐다. 여기서 100은 임베딩의 차원이다.

LSTM 레이어는 2개의 hidden 변수와 임베디드 레이어의 출력을 입력 받는다.

여기서 두 hidden 변수는 임베딩 출력과 동일한 유형이어야 하며, 그 크기는 (num_layers, batch_size, hidden_size)이다.

LSTM은 전체 시퀀스의 데이터를 처리하고 (sequence_length, batch_size, hidden_size) 형상의 출력을 생성한다.

여기서 각 시퀀스 인덱스는 해당 시퀀스의 출력을 나타낸다.

이 경우, 마지막 시퀀스의 출력을 가져온다.

이 시퀀스의 형상은 (batch_size, hidden_dim)이고 이 출력을 선형 레이어에 전달해 출력 번주에 대응시킨다.

마지막으로 모델이 과대적합이 될 경우를 대비해 드롭아웃 레이어를 추가한다.

In [31]:
class IMDBLstm(nn.Module):
    
    def __init__(self,vocab,hidden_size,n_cat,bs=1,nl=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.nl = nl
        self.emb = nn.Embedding(n_vocab,hidden_size)
        self.rnn = nn.LSTM(hidden_size,hidden_size,nl)
        self.fc2 = nn.Linear(hidden_size,n_cat)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self,inp):
        bs = inp.size()[1]
        if bs != self.bs:
            self.bs = bs
        e_out = self.emb(inp)
        h0 = c0 = Variable(e_out.data.new(*(self.nl, self.bs, self.hidden_size)).zero_())
        rnn_o,_ = self.rnn(e_out,(h0,c0)) 
        rnn_o = rnn_o[-1]
        fc = F.dropout(self.fc2(rnn_o),p=0.5)
        return self.softmax(fc)

In [32]:
n_vocab = len(TEXT.vocab)
n_hidden = 100

In [33]:
len(train_iter.dataset)

25000

In [34]:
model = IMDBLstm(n_vocab,n_hidden,3,bs=32)
model = model.cuda()

In [35]:
optimizer = optim.Adam(model.parameters(),lr=1e-3)

# 모델 학습시키기

In [36]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if torch.cuda.is_available():
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [37]:
%%time

train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,5):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)


training loss is  0.88 and training accuracy is 10880/25000     43.52
validation loss is  0.87 and validation accuracy is 11010/25000     44.04
training loss is  0.87 and training accuracy is 11189/25000     44.76
validation loss is  0.87 and validation accuracy is 10925/25000      43.7
training loss is  0.85 and training accuracy is 11520/25000     46.08
validation loss is  0.87 and validation accuracy is 11521/25000     46.08
training loss is  0.81 and training accuracy is 12386/25000     49.54
validation loss is   0.8 and validation accuracy is 12795/25000     51.18
Wall time: 1min 46s


In [38]:
%%time
for epoch in range(1,5):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is   0.7 and training accuracy is 14020/25000     56.08
validation loss is  0.76 and validation accuracy is 13618/25000     54.47
training loss is  0.62 and training accuracy is 14852/25000     59.41
validation loss is  0.68 and validation accuracy is 14296/25000     57.18
training loss is  0.55 and training accuracy is 15583/25000     62.33
validation loss is  0.68 and validation accuracy is 14487/25000     57.95
training loss is  0.51 and training accuracy is 15849/25000      63.4
validation loss is  0.72 and validation accuracy is 14435/25000     57.74
Wall time: 1min 48s


In [ ]:
# 책을 보면 이렇게만 했을 때 정확도 84% 나온다고 되어 있는데 내꺼는 왜 이렇게 나오는지 모르겠다...